In [1]:
import sys
sys.path.append("..")

In [2]:
import bytelevel

In [3]:
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
import numpy as np
from numpy.linalg import norm
import pickle

In [4]:
from keras.layers import Embedding, Conv1D, Input, GRU, LSTM, Bidirectional, Dense, UpSampling1D, Dropout, TimeDistributed, RepeatVector
from keras.models import Model

Using TensorFlow backend.


In [5]:
from hyper_params import *
import text_encoder as te
import text_decoder as td
from data_set import *

In [6]:
with open('../data/norvig/big.txt') as f:
    data = f.read()

In [7]:
chunks = make_chunks(data)

In [8]:
r = np.random.RandomState(42)

In [9]:
r.shuffle(chunks)
train_text, test_text = make_train_test(chunks)

In [19]:
embed_h = EmbeddingHyper(256, 64)
conv_h = ConvHyper(128, 6, 4)
rnn_h = RnnHyper(256, is_lstm=False, is_bidirectional=True, return_sequences=False)
encoder_h = te.Hyper(embed_h, [conv_h, rnn_h])

In [20]:
dernn_h = RnnHyper(64, is_lstm=False, is_bidirectional=False, return_sequences=True, unroll=True)
dec_h = DeconvHyper(128, 6, 4)
decoder_h = td.Hyper(256, [dernn_h, dec_h])

In [21]:
encoder = encoder_h.make_layer()
decoder = decoder_h.make_layer()

In [22]:
def make_model(max_len):
    x = Input(shape=(max_len,), name='text_input')
    h = encoder(x)
    h = decoder(h, max_len)
    model = Model(x, h)
    model.compile(optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy'])

    return model

In [23]:
model64, model128, model256 = make_model(64), make_model(128), make_model(256)

In [24]:
models = {64: model64, 128: model128, 256: model256 }

In [16]:
def training_round(max_len, history=None):
    if history is None:
        initial_epoch = 0
    else:
        initial_epoch = len(history['loss'])
    train, test = SlicedData.Random(train_text, test_text, max_len, 10000, r)
    model = models[max_len]
    newhistory = model.fit(x=train.x, y=train.y,
                            epochs=initial_epoch+5, batch_size=100,
                            validation_data=(test.x, test.y),
                            initial_epoch=initial_epoch)
    if history is None:
        history = newhistory.history
    else:
        history = {key:history[key] + newhistory.history[key] for key in history.keys()}
    return history

In [25]:
history = None

In [26]:
for i in range(500):
    print(i)
    history = training_round(64, history)

0
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 12s - loss: 3.4496 - categorical_accuracy: 0.1515 - val_loss: 3.1122 - val_categorical_accuracy: 0.1617
Epoch 2/5
10000/10000 [==============================] - 11s - loss: 3.1191 - categorical_accuracy: 0.1600 - val_loss: 3.0871 - val_categorical_accuracy: 0.1617
Epoch 3/5
10000/10000 [==============================] - 11s - loss: 3.0979 - categorical_accuracy: 0.1601 - val_loss: 3.0794 - val_categorical_accuracy: 0.1616
Epoch 4/5
10000/10000 [==============================] - 12s - loss: 3.0905 - categorical_accuracy: 0.1601 - val_loss: 3.0739 - val_categorical_accuracy: 0.1617
Epoch 5/5
10000/10000 [==============================] - 12s - loss: 3.0823 - categorical_accuracy: 0.1601 - val_loss: 3.0673 - val_categorical_accuracy: 0.1617
1
Train on 10000 samples, validate on 1000 samples
Epoch 6/10
10000/10000 [==============================] - 12s - loss: 3.0769 - categorical_ac

KeyboardInterrupt: 

In [91]:
embed_h = EmbeddingHyper(256, 64)
conv_h = ConvHyper(128, 6, 4)
rnn_h = RnnHyper(256, is_lstm=False, is_bidirectional=True, return_sequences=False)
dec_h = DeconvHyper(128, 6, 4)

emb = embed_h.make_layer('')
cnn0 = conv_h.make_layer('cnn0')
cnn1 = conv_h.make_layer('cnn1')
cnn2 = conv_h.make_layer('cnn2')
rnn = rnn_h.make_layer('encoder_rnn')

dernn = GRU(256, return_sequences=True, unroll=True)
dec0, up = dec_h.make_layers('dcnn0')
dec1, _ = dec_h.make_layers('dcnn1')
dec2, _ = dec_h.make_layers('dcnn2')
dense = Dense(256, activation='softmax', name='probs')

In [92]:
max_len = 256
x = Input(shape=(max_len,), name='text_input')
h = emb(x)
print(h.shape)
h = cnn0(h)
print(h.shape)
h = cnn1(h)
print(h.shape)
h = cnn2(h)
print(h.shape)
h = rnn(h)
print(h.shape)
h = RepeatVector(int(max_len / 64))(h)
print(h.shape)
h = dernn(h)
print(h.shape)
h = up(h)
print(h.shape)
h = dec0(h)
print(h.shape)
h = up(h)
print(h.shape)
h = dec1(h)
print(h.shape)
h = up(h)
print(h.shape)
h = dec2(h)
print(h.shape)
h = TimeDistributed(dense)(h)
print(h.shape)
model = Model(x, h)
model.compile(optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy'])


(?, 256, 64)
(?, 64, 128)
(?, 16, 128)
(?, 4, 128)
(?, 512)
(?, 4, 512)
(?, 4, 256)
(?, 16, 256)
(?, 16, 128)
(?, 64, 128)
(?, 64, 128)
(?, 256, 128)
(?, 256, 128)
(?, 256, 256)


In [93]:
def make_model(max_len):
    x = Input(shape=(max_len,), name='text_input')
    h = emb(x)
    print(h.shape)
    h = cnn0(h)
    print(h.shape)
    h = cnn1(h)
    print(h.shape)
    h = cnn2(h)
    print(h.shape)
    h = rnn(h)
    print(h.shape)
    h = RepeatVector(int(max_len / 64))(h)
    print(h.shape)
    h = dernn(h)
    print(h.shape)
    h = up(h)
    print(h.shape)
    h = dec0(h)
    print(h.shape)
    h = up(h)
    print(h.shape)
    h = dec1(h)
    print(h.shape)
    h = up(h)
    print(h.shape)
    h = dec2(h)
    print(h.shape)
    h = TimeDistributed(dense)(h)
    print(h.shape)
    model = Model(x, h)
    model.compile(optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy'])
    return model

In [94]:
model64, model128, model256 = make_model(64), make_model(128), make_model(256)

(?, 64, 64)
(?, 16, 128)
(?, 4, 128)
(?, 1, 128)
(?, 512)
(?, 1, 512)
(?, 1, 256)
(?, 4, 256)
(?, 4, 128)
(?, 16, 128)
(?, 16, 128)
(?, 64, 128)
(?, 64, 128)
(?, 64, 256)
(?, 128, 64)
(?, 32, 128)
(?, 8, 128)
(?, 2, 128)
(?, 512)
(?, 2, 512)
(?, 2, 256)
(?, 8, 256)
(?, 8, 128)
(?, 32, 128)
(?, 32, 128)
(?, 128, 128)
(?, 128, 128)
(?, 128, 256)
(?, 256, 64)
(?, 64, 128)
(?, 16, 128)
(?, 4, 128)
(?, 512)
(?, 4, 512)
(?, 4, 256)
(?, 16, 256)
(?, 16, 128)
(?, 64, 128)
(?, 64, 128)
(?, 256, 128)
(?, 256, 128)
(?, 256, 256)


In [95]:
models = {64: model64, 128: model128, 256: model256 }

In [96]:
def check_weights(m0, m1):
    print([norm(v - u) for u, v in zip(m0.get_weights(), m1.get_weights())])

In [100]:
def training_round(max_len, history=None):
    if history is None:
        initial_epoch = 0
    else:
        initial_epoch = len(history['loss'])
    train, test = SlicedData.Random(train_text, test_text, max_len, 10000, r)
    model = models[max_len]
    newhistory = model.fit(x=train.x, y=train.y,
                            epochs=initial_epoch+5, batch_size=32,
                            validation_data=(test.x, test.y),
                            initial_epoch=initial_epoch)
    if history is None:
        history = newhistory.history
    else:
        history = {key:history[key] + newhistory.history[key] for key in history.keys()}
    return history

In [99]:
max_len = 256
train, test = SlicedData.Random(train_text, test_text, max_len, 10000, r)
history = model256.fit(x=train.x, 
        y=train.y,
        epochs=5, batch_size=50,
        validation_data=(test.x, test.y))
history = history.history

Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 30s - loss: 3.1259 - categorical_accuracy: 0.1590 - val_loss: 3.0836 - val_categorical_accuracy: 0.1626
Epoch 2/5
10000/10000 [==============================] - 28s - loss: 3.1060 - categorical_accuracy: 0.1590 - val_loss: 3.0737 - val_categorical_accuracy: 0.1626
Epoch 3/5
10000/10000 [==============================] - 28s - loss: 3.0922 - categorical_accuracy: 0.1590 - val_loss: 3.0601 - val_categorical_accuracy: 0.1625
Epoch 4/5
10000/10000 [==============================] - 28s - loss: 3.0783 - categorical_accuracy: 0.1590 - val_loss: 3.0525 - val_categorical_accuracy: 0.1627
Epoch 5/5
10000/10000 [==============================] - 28s - loss: 3.0697 - categorical_accuracy: 0.1591 - val_loss: 3.0464 - val_categorical_accuracy: 0.1627


In [101]:
for i in range(10):
    print(i)
    history = training_round(256, history)

0
Train on 10000 samples, validate on 1000 samples
Epoch 6/10
10000/10000 [==============================] - 34s - loss: 3.0613 - categorical_accuracy: 0.1602 - val_loss: 3.0410 - val_categorical_accuracy: 0.1628
Epoch 7/10
10000/10000 [==============================] - 31s - loss: 3.0467 - categorical_accuracy: 0.1604 - val_loss: 3.0299 - val_categorical_accuracy: 0.1630
Epoch 8/10
10000/10000 [==============================] - 31s - loss: 3.0326 - categorical_accuracy: 0.1616 - val_loss: 3.0135 - val_categorical_accuracy: 0.1651
Epoch 9/10
10000/10000 [==============================] - 31s - loss: 3.0183 - categorical_accuracy: 0.1635 - val_loss: 3.0058 - val_categorical_accuracy: 0.1666
Epoch 10/10
10000/10000 [==============================] - 31s - loss: 3.0050 - categorical_accuracy: 0.1656 - val_loss: 2.9925 - val_categorical_accuracy: 0.1685
1
Train on 10000 samples, validate on 1000 samples
Epoch 11/15
10000/10000 [==============================] - 32s - loss: 3.0014 - categor

In [104]:
for i in range(80):
    print(i)
    history = training_round(256, history)

0
Train on 10000 samples, validate on 1000 samples
Epoch 56/60
10000/10000 [==============================] - 31s - loss: 2.8253 - categorical_accuracy: 0.1922 - val_loss: 2.8226 - val_categorical_accuracy: 0.1945
Epoch 57/60
10000/10000 [==============================] - 31s - loss: 2.8134 - categorical_accuracy: 0.1940 - val_loss: 2.8195 - val_categorical_accuracy: 0.1948
Epoch 58/60
10000/10000 [==============================] - 31s - loss: 2.8086 - categorical_accuracy: 0.1948 - val_loss: 2.8193 - val_categorical_accuracy: 0.1954
Epoch 59/60
10000/10000 [==============================] - 31s - loss: 2.8055 - categorical_accuracy: 0.1952 - val_loss: 2.8198 - val_categorical_accuracy: 0.1954
Epoch 60/60
10000/10000 [==============================] - 31s - loss: 2.8010 - categorical_accuracy: 0.1960 - val_loss: 2.8156 - val_categorical_accuracy: 0.1957
1
Train on 10000 samples, validate on 1000 samples
Epoch 61/65
10000/10000 [==============================] - 32s - loss: 2.8175 - cat

10000/10000 [==============================] - 31s - loss: 2.6795 - categorical_accuracy: 0.2346 - val_loss: 2.7012 - val_categorical_accuracy: 0.2309
Epoch 104/105
10000/10000 [==============================] - 31s - loss: 2.6710 - categorical_accuracy: 0.2370 - val_loss: 2.6993 - val_categorical_accuracy: 0.2313
Epoch 105/105
10000/10000 [==============================] - 31s - loss: 2.6660 - categorical_accuracy: 0.2383 - val_loss: 2.6996 - val_categorical_accuracy: 0.2309
10
Train on 10000 samples, validate on 1000 samples
Epoch 106/110
10000/10000 [==============================] - 31s - loss: 2.6862 - categorical_accuracy: 0.2341 - val_loss: 2.7015 - val_categorical_accuracy: 0.2309
Epoch 107/110
10000/10000 [==============================] - 31s - loss: 2.6712 - categorical_accuracy: 0.2376 - val_loss: 2.6933 - val_categorical_accuracy: 0.2338
Epoch 108/110
10000/10000 [==============================] - 32s - loss: 2.6632 - categorical_accuracy: 0.2393 - val_loss: 2.6900 - val_c

10000/10000 [==============================] - 31s - loss: 2.4061 - categorical_accuracy: 0.3013 - val_loss: 2.4456 - val_categorical_accuracy: 0.2931
28
Train on 10000 samples, validate on 1000 samples
Epoch 196/200
10000/10000 [==============================] - 31s - loss: 2.4323 - categorical_accuracy: 0.2960 - val_loss: 2.4420 - val_categorical_accuracy: 0.2929
Epoch 197/200
10000/10000 [==============================] - 31s - loss: 2.4142 - categorical_accuracy: 0.2996 - val_loss: 2.4513 - val_categorical_accuracy: 0.2919
Epoch 198/200
10000/10000 [==============================] - 31s - loss: 2.4087 - categorical_accuracy: 0.3009 - val_loss: 2.4452 - val_categorical_accuracy: 0.2935
Epoch 199/200
10000/10000 [==============================] - 31s - loss: 2.4045 - categorical_accuracy: 0.3019 - val_loss: 2.4497 - val_categorical_accuracy: 0.2929
Epoch 200/200
10000/10000 [==============================] - 31s - loss: 2.3977 - categorical_accuracy: 0.3034 - val_loss: 2.4481 - val_c

10000/10000 [==============================] - 31s - loss: 2.3491 - categorical_accuracy: 0.3155 - val_loss: 2.3676 - val_categorical_accuracy: 0.3115
Epoch 288/290
10000/10000 [==============================] - 31s - loss: 2.3417 - categorical_accuracy: 0.3172 - val_loss: 2.3668 - val_categorical_accuracy: 0.3119
Epoch 289/290
10000/10000 [==============================] - 31s - loss: 2.3392 - categorical_accuracy: 0.3175 - val_loss: 2.3673 - val_categorical_accuracy: 0.3124
Epoch 290/290
10000/10000 [==============================] - 31s - loss: 2.3370 - categorical_accuracy: 0.3178 - val_loss: 2.3670 - val_categorical_accuracy: 0.3106
47
Train on 10000 samples, validate on 1000 samples
Epoch 291/295
10000/10000 [==============================] - 31s - loss: 2.3588 - categorical_accuracy: 0.3131 - val_loss: 2.3756 - val_categorical_accuracy: 0.3093
Epoch 292/295
10000/10000 [==============================] - 31s - loss: 2.3417 - categorical_accuracy: 0.3168 - val_loss: 2.3676 - val_c

10000/10000 [==============================] - 31s - loss: 2.3038 - categorical_accuracy: 0.3259 - val_loss: 2.3832 - val_categorical_accuracy: 0.3077
Epoch 380/380
10000/10000 [==============================] - 31s - loss: 2.3030 - categorical_accuracy: 0.3257 - val_loss: 2.3374 - val_categorical_accuracy: 0.3185
65
Train on 10000 samples, validate on 1000 samples
Epoch 381/385
10000/10000 [==============================] - 31s - loss: 2.3206 - categorical_accuracy: 0.3223 - val_loss: 2.3230 - val_categorical_accuracy: 0.3222
Epoch 382/385
10000/10000 [==============================] - 31s - loss: 2.3086 - categorical_accuracy: 0.3251 - val_loss: 2.3652 - val_categorical_accuracy: 0.3116
Epoch 383/385
10000/10000 [==============================] - 31s - loss: 2.3043 - categorical_accuracy: 0.3257 - val_loss: 2.3315 - val_categorical_accuracy: 0.3196
Epoch 384/385
10000/10000 [==============================] - 31s - loss: 2.3018 - categorical_accuracy: 0.3262 - val_loss: 2.3289 - val_c

In [ ]:
for i in range(80):
    print(i)
    history = training_round(256, history)

In [102]:
test.text[:4]

['oss of St. George, awarded to soldiers for bravery in\naction, and in the company of well-known, elderly, and respected\nracing men was training a trotter of his own for a race. He knew a\nlady on one of the boulevards whom he visited of an evening. He led\nth',
 ' he frowned at his wife, the officers\ngrew still merrier, and some of them could not refrain from\nlaughter, for which they hurriedly sought plausible pretexts. When\nhe had gone, taking his wife with him, and had settled down with her\nin their covered cart,',
 'im that would explain to\nher what had happened and to which she could find no answer.\n\n"Natalie, just a word, only one!" he kept repeating, evidently not\nknowing what to say and he repeated it till Helene came up to them.\n\nHelene returned with Natasha to t',
 "was growing lighter and lighter. That curly\ngrass which always grows by country roadsides became clearly\nvisible, still wet with the night's rain; the drooping branches of the\nbirches, also wet, swa

In [103]:
bytelevel.prediction2str(model256.predict(test.x[:4]))

['      iee.                                         he\nsoee                 ee        ee " er                       tes\naee e                                            e eies. Ie  e     \n                                                                     ',
 '                 hhh           ee ies\nto  ee                          hheeeeeeeee              oe\naer e         ttthhhhhe                                eeiis. The  eeee                                                          e\n\n\n\n\n\ne                     ',
 '                     ind\nhh                  ll nn    thhhhh                    tiee.\n\n"Io                          ie.                               he\n\naeaae                                                                                                 ',
 '                            eee. Hhe  eeeeee                                                          he\npere                 ttthhhh                                       iee\nan                   

In [105]:
bytelevel.prediction2str(model256.predict(test.x[:4]))

[' ee seen.. Ae  ee  eeeeeed oo ceatiies aoe seeaeed oe\nceneie, and on the rereent an tee,-aniees an eeee, and sereen ee\ncotiee aot ane cartinee of eereld ao his ane  os ahtie.. Pe  een e\n\na  eee                                                               ',
 'the eeeeees an his aaie, the serteee,\nten  eeate ceeteee, and aone of the  aeite wad cettinn thnt\nhane iee, fan whith  het has aeite seeiet so seenee coreeens. I\nn  ee wod thaed aating his see                                    e e                         ',
 'et theh shaed aereaee to\nhos waed aod eeteeeed the ie whinh the woted heey oo  oteee.\n\n"Ie teeed aaed a saeed haed hoe?" he want aereeteee, ah eeeted woe\nhen  en that ie wat  otthe serieted ao                                                                ',
 'iee sheeing hos aa  an  tet eee. The eeeen,\nthaed whatd an iee aeeae to ce eiee te eeeeed cerele ao thee\nceseete, aeeet wos waon the wighere feiee the setining te eeee, an eed\nwe thire aoee to         

In [106]:
model256.save('../models/model256.h5')